# The Graph and Database Framework

## Basic Usage

The basic building block of the framework is
the **ProgramNode** which is an abstract class that describes a generic node in the graph.
Two basic types of nodes are **PyNode** and **QuaNode**, those contain Python and Qua programs, respectively.
To start a new node we need to give it a label, a program to be executed, define input/output variables. For example: 

In [18]:
#First some imports
from qualibs.graph import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import LoopbackInterface
from qm import SimulationConfig

Let's say we want our node to take a number and output it's square

In [19]:
a = PyNode('square')
a.program = lambda x: {'y':x**2}
a.input_vars = {'x': 3}
a.output_vars = {'y'}
await a.run_async()


RUNNING PyNode <square>...
DONE running node <square>


Here we used the **await** keyword, which is native in python 3.8 and is used to run code asynchronously, that's how the graph framwork works under the hood. Since this tutorial is written in a Jupyter Notebook one must use the asynchronous version explicitly. The simpler version on PyCharm, for example, would be **a.run()**.
Now, let's explain what we did: 
1. We defined the name/label of the node by a string 'square'.
2. The program must define a dictionary as the output, giving names to all the output variables.
3. The input_vars are defined in a keyword arguments format to be given to the program.
4. The output_vars are a set which contains the variables we would like to store in the result


In [20]:
a.result

{'y': 9}

Another possibility of defining a node in one line:

In [21]:
def add(x, y):
    return {"x":x ,"y": y, "sum": x+y}

sum_node = PyNode("sum_two",add,{'x':2,'y':-3},{'sum','x'})

await sum_node.run_async()
sum_node.result


RUNNING PyNode <sum_two>...
DONE running node <sum_two>


{'sum': -1, 'x': 2}

A nice property of the input_vars attribute is that it can be set using direct assignment by the name of the variable, for example:

In [22]:
sum_node.input_vars.x = 5
sum_node.input_vars.y = 6
await sum_node.run_async()
sum_node.result


RUNNING PyNode <sum_two>...
DONE running node <sum_two>


{'sum': 11, 'x': 5}

Notice: that the **input vars** must match the variables in the **signature** of the program, and the **output vars** the **output dictionary** names.

Now let's create a QuaNode! A QuaNode shares the same attributes as the PyNode but has some additional properties:
1. quantum_machine : defines the QuantumMachine to be used to run the Qua program
2. simulation/execution arguments
3. Whether to simulate_or_execute toggle
4. The program of the QuaNode has somewhat different structure

In [23]:
#import a template config
from qualibs.templates.vanilla_config import config

# open a QM
QMm = QuantumMachinesManager()
QM = QMm.open_qm(config)
sim_args = {'simulate': SimulationConfig(int(1e3))}

2020-11-02 10:26:27,390 - qm - INFO - Performing health check
2020-11-02 10:26:27,395 - qm - INFO - Health check passed
2020-11-02 10:26:27,397 - qm - WARNING - Client's version (0.5.124) does not match the server version (0.5.145).


Now, to define a program for the QuaNode, one must wrap a Qua program in a python function, and the python must return the desired qua program that needs to be executed:

In [24]:
def qua_prog(x):
    
    with program() as prog:
        res = declare(int)
        res_str = declare_stream()
        play('playOp', 'qe1')
        assign(res, x)
        save(res, res_str)
        with stream_processing():
            res_str.save_all('res')
    
    return prog

b = QuaNode('save_x',qua_prog,{'x':5},{'res'})

b.simulation_kwargs = sim_args
b.quantum_machine = QM

await b.run_async()
b.result


SIMULATING QuaNode <save_x>...
2020-11-02 10:26:27,920 - qm - INFO - Flags: 
2020-11-02 10:26:27,921 - qm - INFO - Executing high level program
DONE running node <save_x>


{'res': array([5], dtype=int64)}

Here, the **input_vars** as before must match the **input of the given program**,
and the **output_vars** must match the  saved **stream_processing variables** by the Qua program.

## The Graph

Another basic enetity in the framework is the **ProgramGraph**. This is what links together the nodes decribed above, and what controls the execution and dataflow between the nodes. Let's define a new graph:


In [25]:
g = ProgramGraph('new_graph')

Now suppose we want our graph to have nodes 'a','sum_node' connected such that the output variable y of 'a' will be used as input vaiable x of 'sum_node'.

In [26]:
sum_node.input_vars.y = a.output('y')

Now simply add the nodes to the graph:

In [27]:
g.add_nodes(a,sum_node)

Now to run the graph the same story about async applies here. The graph will be run in an order defined by the dependencies, where independent nodes could run asynchronously.

In [28]:
await g.run_async()


RUNNING PyNode <square>...
DONE running node <square>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>


The result is stored in each of the nodes.

In [29]:
sum_node.result

{'sum': 14, 'x': 5}

Let's add node 'b' to the graph, and use the output of 'a' as the input to node 'b'

In [30]:
b.input_vars.x = a.output("y")

g.add_nodes(b)

await g.run_async()


RUNNING PyNode <square>...
DONE running node <square>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>

SIMULATING QuaNode <save_x>...
2020-11-02 10:26:34,764 - qm - INFO - Flags: 
2020-11-02 10:26:34,765 - qm - INFO - Executing high level program
DONE running node <save_x>


In [31]:
b.result

{'res': array([9], dtype=int64)}

We can see the edges of the graph are constructed automatically from the output flow, but one can add edges manually to ensure a time ordered execution of the nodes. For example, since there's no data flow between 'b' and 'sum_node' one can add an edge from 'b' to 'sum_node' to ensure that 'b' runs before 'sum_node'.

In [32]:
g.add_edges((b,sum_node))
await g.run_async()


RUNNING PyNode <square>...
DONE running node <square>

SIMULATING QuaNode <save_x>...
2020-11-02 10:26:39,704 - qm - INFO - Flags: 
2020-11-02 10:26:39,706 - qm - INFO - Executing high level program
DONE running node <save_x>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>


Now, to vizualize the graph we can use the graph DOT format, and put the string into http://www.webgraphviz.com/ 

In [33]:
print(g.export_dot_graph())

digraph new_graph {{square [shape=box];sum_two [shape=box];save_x [shape=ellipse];};"square" -> "sum_two" [label="['y']"];"square" -> "save_x" [label="['y']"];"sum_two";"save_x" -> "sum_two" [label="!none"];}
